In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Initialize

In [17]:
import dspy

In [18]:
# Clear DSPy's global disk cache
if hasattr(dspy, 'cache') and hasattr(dspy.cache, 'disk_cache'):
    dspy.cache.disk_cache.clear()
    print("clear")

clear


In [19]:
import boto3
from typing import Any
from package.base import DriverLM, ModelResponse, Usage


def bedrock_request_fn(messages: list[dict[str, Any]], temperature: float = 0.0, **kwargs) -> dict:
    """
    Request function for AWS Bedrock Converse API.
    
    DSPy passes messages in OpenAI format:
    [
        {"role": "system", "content": "..."},
        {"role": "user", "content": "..."},
        {"role": "assistant", "content": "..."},
        ...
    ]
    
    Bedrock Converse API expects:
    - system: list of system messages (separate parameter)
    - messages: list of user/assistant turns (no system role)
    """
    
    client = boto3.client('bedrock-runtime', region_name='us-east-1')
    
    # Separate system messages from conversation
    system_messages = []
    conversation_messages = []
    
    for msg in messages:
        if msg["role"] == "system":
            system_messages.append({"text": msg["content"]})
        else:
            conversation_messages.append({
                "role": msg["role"],
                "content": [{"text": msg["content"]}]
            })
    
    # Build request
    request_params = {
        "modelId": "us.amazon.nova-micro-v1:0",
        "messages": conversation_messages,
        "inferenceConfig": {
            "temperature": temperature,
            "maxTokens": kwargs.get("max_tokens", 2048),
        }
    }
    
    # Add system messages if present
    if system_messages:
        request_params["system"] = system_messages
    
    # Call Bedrock
    response = client.converse(**request_params)
    
    return response


def bedrock_output_fn(response: dict) -> ModelResponse:
    """
    Parse Bedrock Converse API response into ModelResponse.
    
    Bedrock response format:
    {
        "output": {
            "message": {
                "role": "assistant",
                "content": [{"text": "..."}]
            }
        },
        "usage": {
            "inputTokens": 100,
            "outputTokens": 50,
            "totalTokens": 150
        },
        "stopReason": "end_turn",
        "metrics": {...}
    }
    """
    
    # Extract content
    content = response["output"]["message"]["content"][0]["text"]
    
    # Extract usage
    usage_data = response.get("usage", {})
    usage = Usage(
        prompt_tokens=usage_data.get("inputTokens", 0),
        completion_tokens=usage_data.get("outputTokens", 0),
        total_tokens=usage_data.get("totalTokens", 0)
    )
    
    # Get model ID from response metadata
    model = response.get("ResponseMetadata", {}).get("HTTPHeaders", {}).get("x-amzn-bedrock-model-id", "bedrock-model")
    
    return ModelResponse.from_text(text=content, usage=usage, model=model)



In [20]:
bedrock_request_fn(messages=[{"role": "user", "content": "hi"}], temperature=0.0)

{'ResponseMetadata': {'RequestId': 'e4a8e47b-13bf-4ac6-b1af-5274112d2f4a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 18 Jan 2026 14:47:29 GMT',
   'content-type': 'application/json',
   'content-length': '343',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'e4a8e47b-13bf-4ac6-b1af-5274112d2f4a'},
  'RetryAttempts': 0},
 'output': {'message': {'role': 'assistant',
   'content': [{'text': "Hello! How can I assist you today? Whether you have a question, need information, or just want to chat, I'm here to help. What's on your mind?"}]}},
 'stopReason': 'end_turn',
 'usage': {'inputTokens': 1, 'outputTokens': 39, 'totalTokens': 40},
 'metrics': {'latencyMs': 294}}

In [21]:
native_lm = dspy.LM(
    model="ollama/llama3.2-vision:11b",
    api_base="http://localhost:11434",
    temperature=0.0
)

# Create Bedrock LM
bedrock_lm = DriverLM(
    request_fn=bedrock_request_fn,
    output_fn=bedrock_output_fn,
    temperature=0.0,
    max_tokens=2048,
    cache=True
)

bedrock_lm.clear_cache()

# lm = native_lm
lm = bedrock_lm

In [22]:
class ChatSignature(dspy.Signature):
    """Use the conversation_context to answer questions about the user."""
    conversation_context: str = dspy.InputField(desc="All previous messages in this conversation")
    user_message: str = dspy.InputField(desc="Current question")
    response: str = dspy.OutputField(desc="Answer using conversation_context")


demos = [
    dspy.Example(
        conversation_context="No previous messages",
        user_message="Hi, my name is Alice",
        response="Hello Alice! Nice to meet you."
    ).with_inputs("conversation_context", "user_message"),
    
    dspy.Example(
        conversation_context="USER: Hi, my name is Alice\nASSISTANT: Hello Alice! Nice to meet you.",
        user_message="What's my name?",
        response="Your name is Alice."  # No greeting
    ).with_inputs("conversation_context", "user_message"),
    
    # Add this demo:
    dspy.Example(
        conversation_context="USER: Hi, my name is Alice\nASSISTANT: Hello Alice!\nUSER: What's my name?\nASSISTANT: Your name is Alice.",
        user_message="How are you?",
        response="I'm doing well, thank you for asking!"  # No greeting
    ).with_inputs("conversation_context", "user_message"),
]


In [23]:
dspy.configure(lm=lm)
qa_zero_shot = dspy.Predict(ChatSignature)
qa_cot = dspy.ChainOfThought(ChatSignature)
qa_zero_shot.demos = demos
qa_cot.predict.demos = demos

In [25]:
qa_zero_shot.predictors()

[Predict(ChatSignature(conversation_context, user_message -> response
     instructions='Use the conversation_context to answer questions about the user.'
     conversation_context = Field(annotation=str required=True json_schema_extra={'desc': 'All previous messages in this conversation', '__dspy_field_type': 'input', 'prefix': 'Conversation Context:'})
     user_message = Field(annotation=str required=True json_schema_extra={'desc': 'Current question', '__dspy_field_type': 'input', 'prefix': 'User Message:'})
     response = Field(annotation=str required=True json_schema_extra={'desc': 'Answer using conversation_context', '__dspy_field_type': 'output', 'prefix': 'Response:'})
 ))]

In [26]:
qa_cot.predictors()

[Predict(StringSignature(conversation_context, user_message -> reasoning, response
     instructions='Use the conversation_context to answer questions about the user.'
     conversation_context = Field(annotation=str required=True json_schema_extra={'desc': 'All previous messages in this conversation', '__dspy_field_type': 'input', 'prefix': 'Conversation Context:'})
     user_message = Field(annotation=str required=True json_schema_extra={'desc': 'Current question', '__dspy_field_type': 'input', 'prefix': 'User Message:'})
     reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_type': 'output'})
     response = Field(annotation=str required=True json_schema_extra={'desc': 'Answer using conversation_context', '__dspy_field_type': 'output', 'prefix': 'Response:'})
 ))]

In [ ]:
conversation_context = """
USER: My name is Bank. Nice to meet you.
ASSISTANT: Hello Bank! It's nice to meet you.
USER: What's my favorite pizza topping?
ASSISTANT: I'm sorry, but I don't have that information. It's best to ask you directly about your favorite pizza topping.
USER: For your previous answer, I love hawaiian!
ASSISTANT: It's great to hear that you love Hawaiian pizza! It's a delicious choice.
USER: If I wanna find my new favorite pizza topping, what would you recommend?
ASSISTANT: That's a fun question! If you're looking to try something new, I'd recommend trying a pizza with truffle oil and mushrooms. The earthy flavor of the mushrooms combined with the richness of truffle oil can create a unique and delicious experience. However, the best topping is always personal preference, so feel free to experiment and find what you love!
"""

params = {
    "conversation_context": conversation_context,
    "user_message": "What's my name?"
}

# dspy.Predict: simple and lean

In [10]:
qa_zero_shot(**params)

Prediction(
    response='Your name is Bank.'
)

In [11]:
dspy.inspect_history()





[2026-01-18T21:29:06.122353]

System message:

Your input fields are:
1. `conversation_context` (str): All previous messages in this conversation
2. `user_message` (str): Current question
Your output fields are:
1. `response` (str): Answer using conversation_context
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## conversation_context ## ]]
{conversation_context}

[[ ## user_message ## ]]
{user_message}

[[ ## response ## ]]
{response}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Use the conversation_context to answer questions about the user.


User message:

[[ ## conversation_context ## ]]
No previous messages

[[ ## user_message ## ]]
Hi, my name is Alice


Assistant message:

[[ ## response ## ]]
Hello Alice! Nice to meet you.

[[ ## completed ## ]]


User message:

[[ ## conversation_context ## ]]
USER: Hi, my name is Alice
ASSISTANT: Hello Alice! Nice to meet you.

[[ ## user_message 

In [12]:
lm.history[-1]

{'prompt': None,
 'messages': [{'role': 'system',
   'content': 'Your input fields are:\n1. `conversation_context` (str): All previous messages in this conversation\n2. `user_message` (str): Current question\nYour output fields are:\n1. `response` (str): Answer using conversation_context\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## conversation_context ## ]]\n{conversation_context}\n\n[[ ## user_message ## ]]\n{user_message}\n\n[[ ## response ## ]]\n{response}\n\n[[ ## completed ## ]]\nIn adhering to this structure, your objective is: \n        Use the conversation_context to answer questions about the user.'},
  {'role': 'user',
   'content': '[[ ## conversation_context ## ]]\nNo previous messages\n\n[[ ## user_message ## ]]\nHi, my name is Alice'},
  {'role': 'assistant',
   'content': '[[ ## response ## ]]\nHello Alice! Nice to meet you.\n\n[[ ## completed ## ]]\n'},
  {'role': 'user',
   'content': "[[ ## conversation_co

# dspy.ChainOfThought: adding reasoning as dspy.OutputField

Note: based on inspect the prompt with demos, it seems dspy.CoT can handle normal demo with CoT as well by using  
`[[ ## reasoning ## ]]\nNot supplied for this particular example. \n\n`  

Hence, dspy.Example can handle missing variables quite well.

In [13]:
qa_cot(**params)

Prediction(
    reasoning='Based on the conversation context, the user has previously introduced themselves as "Bank". Therefore, the user\'s name is Bank.',
    response='Your name is Bank.'
)

In [14]:
dspy.inspect_history()





[2026-01-18T21:29:07.531635]

System message:

Your input fields are:
1. `conversation_context` (str): All previous messages in this conversation
2. `user_message` (str): Current question
Your output fields are:
1. `reasoning` (str): 
2. `response` (str): Answer using conversation_context
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## conversation_context ## ]]
{conversation_context}

[[ ## user_message ## ]]
{user_message}

[[ ## reasoning ## ]]
{reasoning}

[[ ## response ## ]]
{response}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Use the conversation_context to answer questions about the user.


User message:

This is an example of the task, though some input or output fields are not supplied.

[[ ## conversation_context ## ]]
No previous messages

[[ ## user_message ## ]]
Hi, my name is Alice


Assistant message:

[[ ## reasoning ## ]]
Not supplied for this particular example. 

[[ #

In [15]:
lm.history[-1]

{'prompt': None,
 'messages': [{'role': 'system',
   'content': 'Your input fields are:\n1. `conversation_context` (str): All previous messages in this conversation\n2. `user_message` (str): Current question\nYour output fields are:\n1. `reasoning` (str): \n2. `response` (str): Answer using conversation_context\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## conversation_context ## ]]\n{conversation_context}\n\n[[ ## user_message ## ]]\n{user_message}\n\n[[ ## reasoning ## ]]\n{reasoning}\n\n[[ ## response ## ]]\n{response}\n\n[[ ## completed ## ]]\nIn adhering to this structure, your objective is: \n        Use the conversation_context to answer questions about the user.'},
  {'role': 'user',
   'content': 'This is an example of the task, though some input or output fields are not supplied.\n\n[[ ## conversation_context ## ]]\nNo previous messages\n\n[[ ## user_message ## ]]\nHi, my name is Alice'},
  {'role': 'assistant',
   